In [6]:
import getpass
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_openai import ChatOpenAI
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

chat model

In [5]:

# if not os.environ.get("DEEPSEEK_API_KEY"):
#   os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

# from langchain.chat_models import init_chat_model

# llm = init_chat_model("deepseek-chat", model_provider="deepseek")
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 聊天模型名称（正确）
    temperature=0,          # 生成随机性，0表示确定性最高
    api_key="sk-5hmjTgNvEwpzxliLy8Ub6SBkjdt5GkotJUcr9Y8HoW8CQ7bX",
    base_url="https://api2.aigcbest.top/v1"
)

embeddings model

In [7]:




from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
embeddings = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 聊天模型名称（正确）
    temperature=0,          # 生成随机性，0表示确定性最高
    api_key="sk-5hmjTgNvEwpzxliLy8Ub6SBkjdt5GkotJUcr9Y8HoW8CQ7bX",
    base_url="https://api2.aigcbest.top/v1"
)
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [9]:


# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

AttributeError: 'ChatOpenAI' object has no attribute 'embed_documents'

In [8]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

NameError: name 'graph' is not defined